# HEADY COLAB PRO+ GPU NODE — ATLAS (The Auto-Archivist)
**Sacred Geometry Architecture** · Embeddings, Semantic Search, Documentation

Registers with HeadyCloud via branded domains only:
- `headysystems.com` · `headycloud.com` · `headyconnection.com`

**Setup:** Add to Colab Secrets (key icon): `HEADY_API_KEY`, `HF_TOKEN`, `NGROK_TOKEN`

In [ ]:
!nvidia-smi
!pip install -q fastapi uvicorn[standard] pyngrok nest_asyncio \
    sentence-transformers transformers accelerate \
    torch aiohttp requests scikit-learn

In [ ]:
import os, json, torch
from datetime import datetime
from google.colab import userdata

HEADY_CONFIG = {
    'cloud_layers': {
        'headysystems': {'url': 'https://headyio.com', 'role': 'production'},
        'headyme': {'url': 'https://headycloud.com', 'role': 'cloud_layer'},
        'headyconnection': {'url': 'https://headyconnection.com', 'role': 'bridge'},
    },
    'heady_api_key': userdata.get('HEADY_API_KEY'),
    'hf_token': userdata.get('HF_TOKEN'),
    'ngrok_token': userdata.get('NGROK_TOKEN'),
    'node_id': 'atlas',
    'node_role': 'The Auto-Archivist',
    'node_type': 'ai-node',
    'cloud_layer': 'colab_pro_plus',
    'primary_tool': 'autodoc',
    'triggers': ['documentation', 'embeddings', 'semantic_search', 'summarize'],
    'capabilities': {
        'gpu': torch.cuda.is_available(),
        'gpu_type': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'none',
        'gpu_memory_gb': round(torch.cuda.get_device_properties(0).total_memory / 1e9, 1) if torch.cuda.is_available() else 0,
        'max_concurrent_tasks': 8,
        'background_execution': True,
        'primary_functions': ['generate_embeddings', 'semantic_search', 'documentation', 'summarize'],
    },
    'port': 8000,
    'heartbeat_interval_sec': 30,
}

print(f'ATLAS NODE | GPU: {HEADY_CONFIG["capabilities"]["gpu_type"]} | VRAM: {HEADY_CONFIG["capabilities"]["gpu_memory_gb"]} GB')
print('Targets: headyio.com | headycloud.com | headyconnection.com')

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline as hf_pipeline

heady_models = {}
heady_models['embeddings'] = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to('cuda')
heady_models['embeddings_large'] = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2').to('cuda')
heady_models['summarizer'] = hf_pipeline('summarization', model='facebook/bart-large-cnn', device=0, torch_dtype=torch.float16)
print('All ATLAS models loaded to GPU')

In [ ]:
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel
from typing import Optional, Dict, Any
from sklearn.metrics.pairwise import cosine_similarity

app = FastAPI(title='Heady ATLAS Node', version='3.0.0')
node_state = {'status': 'initializing', 'tasks_completed': 0, 'tasks_failed': 0, 'uptime_start': datetime.now().isoformat(), 'current_tasks': [], 'registered_with': []}

class HeadyTask(BaseModel):
    task_id: str
    task_type: str
    payload: Dict[str, Any]
    priority: str = 'P1'
    source_cloud: str = 'unknown'

@app.get('/health')
async def health():
    gpu_mem = torch.cuda.memory_allocated(0) / 1e9 if torch.cuda.is_available() else 0
    return {'node_id': 'atlas', 'node_role': 'The Auto-Archivist', 'status': node_state['status'], 'gpu': {'type': HEADY_CONFIG['capabilities']['gpu_type'], 'used_gb': round(gpu_mem, 2), 'total_gb': HEADY_CONFIG['capabilities']['gpu_memory_gb']}, 'tasks_completed': node_state['tasks_completed'], 'current_load': len(node_state['current_tasks']), 'triggers': HEADY_CONFIG['triggers']}

@app.post('/task')
async def execute_task(task: HeadyTask, authorization: Optional[str] = Header(None)):
    if not authorization or authorization != f'Bearer {HEADY_CONFIG["heady_api_key"]}': raise HTTPException(401, 'Invalid API key')
    start = datetime.now()
    try:
        node_state['current_tasks'].append(task.task_id)
        handlers = {'generate_embeddings': do_embeddings, 'semantic_search': do_search, 'documentation': do_docs, 'summarize': do_summarize}
        result = handlers.get(task.task_type, lambda p: (_ for _ in ()).throw(ValueError(f'Unknown: {task.task_type}')))(task.payload)
        node_state['tasks_completed'] += 1; node_state['current_tasks'].remove(task.task_id)
        return {'task_id': task.task_id, 'status': 'success', 'result': result, 'execution_time_ms': (datetime.now() - start).total_seconds() * 1000, 'gpu_accelerated': True}
    except Exception as e:
        node_state['tasks_failed'] += 1
        if task.task_id in node_state['current_tasks']: node_state['current_tasks'].remove(task.task_id)
        return {'task_id': task.task_id, 'status': 'failed', 'error': str(e)}

def do_embeddings(p):
    m = heady_models['embeddings_large'] if p.get('large_model') else heady_models['embeddings']
    e = m.encode(p.get('texts', []), batch_size=64, device='cuda', convert_to_numpy=True)
    return {'embeddings': e.tolist(), 'dimensions': e.shape[1], 'count': len(p.get('texts', []))}

def do_search(p):
    q = heady_models['embeddings'].encode([p['query']], device='cuda')
    d = heady_models['embeddings'].encode(p.get('documents', []), batch_size=64, device='cuda')
    s = cosine_similarity(q, d)[0]; top = s.argsort()[-p.get('top_k', 5):][::-1]
    return {'query': p['query'], 'results': [{'document': p['documents'][i], 'score': float(s[i]), 'rank': r+1} for r, i in enumerate(top)]}

def do_docs(p):
    code = p.get('code', ''); lang = p.get('language', 'python')
    if len(code) > 500: return {'documentation': heady_models['summarizer'](code, max_length=150, min_length=50, do_sample=False)[0]['summary_text']}
    return {'documentation': f'```{lang}\n{code}\n```'}

def do_summarize(p):
    s = heady_models['summarizer'](p['text'], max_length=p.get('max_length', 150), min_length=p.get('min_length', 50), do_sample=False)
    return {'summary': s[0]['summary_text'], 'compression_ratio': len(s[0]['summary_text']) / len(p['text'])}

print('FastAPI + 4 GPU task handlers ready')

In [ ]:
import asyncio, aiohttp, threading
from pyngrok import ngrok
import nest_asyncio, uvicorn
nest_asyncio.apply()

# Authenticate ngrok — hardcoded fallback if Colab Secrets not set
try:
    ngrok_token = HEADY_CONFIG['ngrok_token']
except:
    ngrok_token = "39ZBirdUD63xgta7yN7OFZpE84m_3QZyJTDno1b8Yhv9Nfy8s"
ngrok.set_auth_token(ngrok_token)
tunnel = ngrok.connect(HEADY_CONFIG['port'], 'http')
PUBLIC_URL = tunnel.public_url
print(f'Tunnel: {PUBLIC_URL}')

async def register_with_clouds():
    reg = {'node_id': 'atlas', 'node_type': 'ai-node', 'node_role': 'The Auto-Archivist', 'cloud_layer': 'colab_pro_plus', 'url': PUBLIC_URL, 'primary_tool': 'autodoc', 'triggers': HEADY_CONFIG['triggers'], 'capabilities': HEADY_CONFIG['capabilities']}
    for name, cloud in HEADY_CONFIG['cloud_layers'].items():
        for attempt in range(3):
            try:
                async with aiohttp.ClientSession() as session:
                    async with session.post(f'{cloud["url"]}/api/nodes/register', json=reg, headers={'Authorization': f'Bearer {HEADY_CONFIG["heady_api_key"]}', 'Content-Type': 'application/json'}, timeout=aiohttp.ClientTimeout(total=30)) as resp:
                        if resp.status == 200:
                            node_state['registered_with'].append(name)
                            print(f'Registered with {name} ({cloud["url"]})')
                            break
                        else: print(f'Registration attempt {attempt+1} failed: HTTP {resp.status}')
            except Exception as e:
                print(f'Could not reach {name} (attempt {attempt+1}): {e}')
                if attempt < 2: await asyncio.sleep(5)
    node_state['status'] = 'active' if node_state['registered_with'] else 'standalone'
    print(f'Registered with {len(node_state["registered_with"])} cloud layer(s)')

async def heartbeat_loop():
    while True:
        for name, cloud in HEADY_CONFIG['cloud_layers'].items():
            if name not in node_state['registered_with']: continue
            try:
                hb = {'node_id': 'atlas', 'status': node_state['status'], 'metrics': {'tasks_completed': node_state['tasks_completed'], 'tasks_failed': node_state['tasks_failed'], 'current_load': len(node_state['current_tasks'])}}
                async with aiohttp.ClientSession() as session:
                    async with session.post(f'{cloud["url"]}/api/nodes/heartbeat', json=hb, headers={'Authorization': f'Bearer {HEADY_CONFIG["heady_api_key"]}'}, timeout=aiohttp.ClientTimeout(total=5)) as resp:
                        if resp.status == 200: pass
            except: pass
        await asyncio.sleep(HEADY_CONFIG['heartbeat_interval_sec'])

asyncio.run(register_with_clouds())
threading.Thread(target=lambda: asyncio.run(heartbeat_loop()), daemon=True).start()
print(f'ATLAS node live at {PUBLIC_URL} — accepting tasks from HeadyCloud')
# Colab VM bind address for uvicorn (dual-stack all-interfaces)
uvicorn.run(app, host='::', port=HEADY_CONFIG['port'])